# Etape 2.1 : Nettoyage des donnees meteos avec Pandas

**Objectif** : Traiter des donnees structurees avec un langage de programmation

**Livrables** :
- Ce notebook `04_nettoyage_meteo_pandas.ipynb`
- Dataset nettoye `output/04_meteo_clean.csv`
- Rapport avant/apres nettoyage (completude par colonne)

---
---

## Imports

In [1]:
import sys
import os
from pathlib import Path
import time
from datetime import datetime
import psutil
import pandas as pd
import numpy as np

---

## (optionnel) Enregistrement de la date de la dernière execution de ce notebook

In [2]:
print(f"- Date de la dernière execution de ce notebook : {datetime.now().strftime('%d/%m/%Y %H:%M:%S')} (FR)")

- Date de la dernière execution de ce notebook : 20/02/2026 20:32:46 (FR)


---

## (optionnel) Mesure du temps de traitement global pour ce script - enregistrement de l'heure de début + estimation instantanée des ressources machine libres

In [3]:
# --- Heure de début
start_time_04 = time.time()

# --- Machine: current available RAM (in GB)
ram_available_04 = psutil.virtual_memory().available / (1024**3)

# --- Machine: current available CPU
logical = psutil.cpu_count()
physical = psutil.cpu_count(logical=False) or logical

cpu_used = psutil.cpu_percent(interval=2)
cpu_available_pct_04 = 100 - cpu_used

available_logical_04 = logical * cpu_available_pct_04 / 100
available_physical_04 = physical * cpu_available_pct_04 / 100

# --- Show available resources
print(f"- Current machine RAM available : {ram_available_04:.2f} GB")
print(f"- Current machine CPU available : {cpu_available_pct_04:.1f}%")
print(f"    Approx logical cores free  : {available_logical_04:.2f}")
print(f"    Approx physical cores free : {available_physical_04:.2f}")

- Current machine RAM available : 10.76 GB
- Current machine CPU available : 84.2%
    Approx logical cores free  : 13.47
    Approx physical cores free : 6.74


---

## Chemins des données

In [4]:
# ==============================================================================================================
#                                                  INPUTS
# ==============================================================================================================
IN_DIR = (Path.cwd() / ".." / "data").resolve()
IN_METEO_RAW_CSV = os.path.join(IN_DIR, "meteo_raw.csv")

# ==============================================================================================================
#                                                OUTPUTS
# ==============================================================================================================
OUT_DIR = (Path.cwd() / ".." / "output").resolve()
OUT_METEO_CLEAN_CSV = os.path.join(OUT_DIR, "04_meteo_clean.csv")

# ==============================================================================================================
#                                                OTHERS
# ==============================================================================================================
TMP_DIR = (Path.cwd() / ".." / "my_tmp").resolve()
TMP_FILE_TXT = TMP_DIR / "tmp_04_resources.txt" # Enregistrer les metrics pour ce script

---

## Chargement du fichier meteo_raw.csv avec Pandas (les données météo brutes)

In [5]:
# --- Chargement des données avec Pandas
df_meteo_raw = pd.read_csv(IN_METEO_RAW_CSV)

# --- Informations
print("df_meteo_raw :")
print(f"- Nombre de lignes, colonnes : {df_meteo_raw.shape}")
print("- Schema :")
df_meteo_raw.info()
print("- Apperçu :")
df_meteo_raw.head(10)

df_meteo_raw :
- Nombre de lignes, colonnes : (252612, 7)
- Schema :
<class 'pandas.DataFrame'>
RangeIndex: 252612 entries, 0 to 252611
Data columns (total 7 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   commune                  252612 non-null  str    
 1   timestamp                252612 non-null  str    
 2   temperature_c            251383 non-null  str    
 3   humidite_pct             252612 non-null  float64
 4   rayonnement_solaire_wm2  252612 non-null  float64
 5   vitesse_vent_kmh         252612 non-null  float64
 6   precipitation_mm         252612 non-null  float64
dtypes: float64(4), str(3)
memory usage: 20.5 MB
- Apperçu :


,commune,timestamp,temperature_c,humidite_pct,rayonnement_solaire_wm2,vitesse_vent_kmh,precipitation_mm
0,Saint-Etienne,09/15/2024 15:00:00,17.1,143.3,244.9,14.3,0.0
1,Bordeaux,21/07/2023 15:00,19.6,50.6,414.9,3.2,0.0
2,Montpellier,2023-09-18 20:00:00,18.3,65.7,218.4,13.6,0.0
3,Le Havre,01/03/2024 22:00:00,3.7,94.9,6.8,18.6,11.6
4,Lille,29/10/2024 20:00,14.0,42.9,781.8,4.0,0.0
5,Bordeaux,22/12/2023 13:00,4.4,36.9,796.4,6.1,0.0
6,Marseille,09/15/2023 21:00:00,22.5,86.8,5.8,32.6,0.0
7,Toulouse,30/05/2023 00:00,8.3,66.3,26.4,31.4,7.2
8,Bordeaux,2024-10-05T09:00:00,11.5,69.8,71.4,34.4,0.0
9,Toulon,2024-09-28T21:00:00,19.2,79.0,13.1,31.8,0.0


---

## Identification et traitement des valeurs manquantes (Diagnostic de la qualite des donnees) :
- Interpolation linéaire pour température et humidité.
- Forward fill pour les precipitation_mm.

In [6]:
def quality_report(df, name="DataFrame"):
    print(f"RAPPORT QUALITE - {name}")
    print(f"- Lignes: {len(df):,}")
    print(f"- Colonnes: {len(df.columns)}")
    
    report = []
    for col in df.columns:
        total = len(df)
        missing = df[col].isna().sum() + (df[col] == '').sum() if df[col].dtype == 'object' else df[col].isna().sum()
        completude = (1 - missing / total) * 100
        unique = df[col].nunique()
        dtype = df[col].dtype
        
        report.append({
            'Colonne': col,
            'Type': str(dtype),
            'Manquants': missing,
            'Completude %': round(completude, 2),
            'Uniques': unique
        })
    
    return pd.DataFrame(report)

quality_report(df_meteo_raw, "Meteo Brute")

RAPPORT QUALITE - Meteo Brute
- Lignes: 252,612
- Colonnes: 7


,Colonne,Type,Manquants,Completude %,Uniques
0,commune,str,0,100.00,15
1,timestamp,str,0,100.00,69066
2,temperature_c,str,1229,99.51,810
3,humidite_pct,float64,0,100.00,1094
4,rayonnement_solaire_wm2,float64,0,100.00,8685
5,vitesse_vent_kmh,float64,0,100.00,401
6,precipitation_mm,float64,0,100.00,151


In [7]:
# --- Examiner les valeurs problematiques dans temperature_c
print("- Valeurs uniques non numeriques dans temperature_c:")
temp_non_numeric = df_meteo_raw[
    ~df_meteo_raw['temperature_c'].astype(str).str.match(r'^-?[0-9]+[.,]?[0-9]*$', na=False)
]['temperature_c'].unique()
print(temp_non_numeric)

- Valeurs uniques non numeriques dans temperature_c:
<ArrowStringArray>
[nan]
Length: 1, dtype: str


In [8]:
# --- Transf. des val. de temperature_c en numerique (ou NaN si valeurs non numerique): 
df_temp = df_meteo_raw.copy()
df_temp['temperature_c'] = pd.to_numeric(
    df_temp['temperature_c'].astype(str).str.replace(',', '.'), 
    errors='coerce' # Toute valeur invalide ("NA", "erreur", "null", vide) => NaN
)

print("- Distribution temperature:")
print(df_temp['temperature_c'].describe())

# --- Valeurs aberrantes de temperature_c
print("\n- Nombres des valeurs aberrantes de temperature :")
print(f"    - Temperatures < -40 : {(df_temp['temperature_c'] < -40).sum()}")
print(f"    - Temperatures > 50 : {(df_temp['temperature_c'] > 50).sum()}")

- Distribution temperature:


count    251383.000000
mean         15.101909
std          10.996268
min         -70.000000
25%           8.700000
50%          14.500000
75%          21.000000
max         100.000000
Name: temperature_c, dtype: float64

- Nombres des valeurs aberrantes de temperature :
    - Temperatures < -40 : 695
    - Temperatures > 50 : 1282


In [9]:
# --- Humidite (humidite_pct) hors bornes
df_temp['humidite_pct'] = pd.to_numeric(df_temp['humidite_pct'], errors='coerce')

print("\n- Distribution humidite:")
print(df_temp['humidite_pct'].describe())

print(f"\n- Humidite > 100% : {(df_temp['humidite_pct'] > 100).sum()}")
print(f"- Humidite < 0% : {(df_temp['humidite_pct'] < 0).sum()}")


- Distribution humidite:
count    252612.00000
mean         63.03070
std          19.49437
min          30.00000
25%          46.50000
50%          62.80000
75%          79.10000
max         150.00000
Name: humidite_pct, dtype: float64

- Humidite > 100% : 1801
- Humidite < 0% : 0


---

## Nettoyage des donnees meteo

In [10]:
def parse_timestamp(ts):
    """Parse les timestamps multi-formats."""
    if pd.isna(ts):
        return pd.NaT
    
    formats = [
        "%Y-%m-%d %H:%M:%S",
        "%d/%m/%Y %H:%M:%S",  
        "%d/%m/%Y %H:%M",
        "%m/%d/%Y %H:%M:%S",
        "%m/%d/%Y %H:%M",     
        "%Y-%m-%dT%H:%M:%S",
    ]
    
    for fmt in formats:
        try:
            return datetime.strptime(str(ts), fmt)
        except ValueError:
            continue
    
    return pd.NaT

df_meteo = df_meteo_raw.copy()

# Parser les timestamps
print("[1/6] Parsing des timestamps ...")
df_meteo['timestamp'] = df_meteo['timestamp'].apply(parse_timestamp)
invalid_ts = df_meteo['timestamp'].isna().sum()
print(f"    - Timestamps invalides: {invalid_ts}")

# Vérifier s’il existe des timestamps non horaires & agreger à l'heure si c'est le cas
print()
mask_not_hourly = (
    (df_meteo["timestamp"].dt.minute != 0) |
    (df_meteo["timestamp"].dt.second != 0)
)
if mask_not_hourly.any(): 
    print("    [warning]: il y a des timestamps avec minutes/secondes => aggregation à l'heure :")
    # Créer une heure 'arrondie'
    df_meteo["ts_h"] = df_meteo["timestamp"].dt.floor("h")
    # Agréger à l’heure
    df_meteo = (
        df_meteo
        .groupby(["commune", "ts_h"], as_index=False)
        .agg({
            "temperature_c": "mean",
            "humidite_pct": "mean",
            "rayonnement_solaire_wm2": "mean",
            "vitesse_vent_kmh": "mean",
            "precipitation_mm": "sum",   
            "jour": "first",
            "mois": "first",
            "saison": "first",
            "jour_de_semaine": "first",
        })
    )
    # pour la clareté : supprimer la colonne timestamp (garder seulement ts_h)
    df_meteo = df_meteo.drop(columns=["timestamp"])
else:
    print("    - [ok]: les timestamps sont 100% horaires, pas besoin d'aggregation")
    
    # pour la clareté : renommer la colonne timestamp par ts_h
    df_meteo = df_meteo.rename(columns={"timestamp": "ts_h"})

# Supprimer les lignes sans ts_h (timestamp) valide
print()
print("    - Suppression des lignes sans ts_h (timestamp) valides")
df_meteo = df_meteo.dropna(subset=['ts_h'])

# Apperçu de df_meteo
print()
print("    - Apperçu de df_meteo :")
df_meteo.head()


[1/6] Parsing des timestamps ...


    - Timestamps invalides: 0

    - [ok]: les timestamps sont 100% horaires, pas besoin d'aggregation

    - Suppression des lignes sans ts_h (timestamp) valides

    - Apperçu de df_meteo :


,commune,ts_h,temperature_c,humidite_pct,rayonnement_solaire_wm2,vitesse_vent_kmh,precipitation_mm
0,Saint-Etienne,2024-09-15 15:00:00,17.1,143.3,244.9,14.3,0.0
1,Bordeaux,2023-07-21 15:00:00,19.6,50.6,414.9,3.2,0.0
2,Montpellier,2023-09-18 20:00:00,18.3,65.7,218.4,13.6,0.0
3,Le Havre,2024-03-01 22:00:00,3.7,94.9,6.8,18.6,11.6
4,Lille,2024-10-29 20:00:00,14.0,42.9,781.8,4.0,0.0


In [11]:
# --- Convertir les colonnes numeriques
print("\n[2/6] Conversion des colonnes numeriques ...")

# --- Temperature (remplacer virgule par point)
df_meteo['temperature_c'] = pd.to_numeric(
    df_meteo['temperature_c'].astype(str).str.replace(',', '.'),
    errors='coerce' # Si une valeur ne peut pas être convertie en nombre, transforme-la en NaN
)

# --- Autres colonnes
df_meteo['humidite_pct'] = pd.to_numeric(df_meteo['humidite_pct'], errors='coerce')
df_meteo['rayonnement_solaire_wm2'] = pd.to_numeric(df_meteo['humidite_pct'], errors='coerce')
df_meteo['vitesse_vent_kmh'] = pd.to_numeric(df_meteo['vitesse_vent_kmh'], errors='coerce')
df_meteo['precipitation_mm'] = pd.to_numeric(df_meteo['precipitation_mm'], errors='coerce')

print("  Conversions effectuees.")


[2/6] Conversion des colonnes numeriques ...
  Conversions effectuees.


In [12]:
# --- Corriger les valeurs aberrantes
print("\n[3/6] Correction des valeurs aberrantes ...")

# Temperatures hors [-40, 50] -> NaN
temp_outliers = ((df_meteo['temperature_c'] < -40) | (df_meteo['temperature_c'] > 50)).sum()
df_meteo.loc[
    (df_meteo['temperature_c'] < -40) | (df_meteo['temperature_c'] > 50),
    'temperature_c'
] = np.nan
print(f"    - Nombre de 'temperatures' aberrantes (hors [-40, 50]) transformées en NaN : {temp_outliers}")

# Humidite hors [0, 100] -> clipper
humidity_outliers = ((df_meteo['humidite_pct'] < 0) | (df_meteo['humidite_pct'] > 100)).sum()
df_meteo['humidite_pct'] = df_meteo['humidite_pct'].clip(0, 100)
print(f"    - Nombre de 'humidite' abérrantes (hors [0, 100]) clippee [0, 100]: {humidity_outliers}")

# Rayonnement solaire negatif -> 0
rayonnement_outliers = ((df_meteo['rayonnement_solaire_wm2'] < 0)).sum()
df_meteo['rayonnement_solaire_wm2'] = df_meteo['rayonnement_solaire_wm2'].clip(lower=0)
print(f"    - Nombre de 'rayonnements solaire' abérrents (< 0) transformées en 0 : {rayonnement_outliers}")


[3/6] Correction des valeurs aberrantes ...
    - Nombre de 'temperatures' aberrantes (hors [-40, 50]) transformées en NaN : 1977
    - Nombre de 'humidite' abérrantes (hors [0, 100]) clippee [0, 100]: 1801
    - Nombre de 'rayonnements solaire' abérrents (< 0) transformées en 0 : 0


In [13]:
# --- Traiter les valeurs manquantes
print("[4/6] Traitement des valeurs manquantes ...")

# Trier par commune et timestamp pour l'interpolation
df_meteo = df_meteo.sort_values(['commune', 'ts_h'])

# Interpolation lineaire pour temperature et humidite ... (par commune) (les autres colonnes au choix) => (Variables continues et progressives)
for col in ['temperature_c', 'humidite_pct', 'rayonnement_solaire_wm2', 'vitesse_vent_kmh']:
    before_na = df_meteo[col].isna().sum()
    df_meteo[col] = df_meteo.groupby('commune')[col].transform(
        lambda x: x.interpolate(method='linear', limit_direction='both')
    )
    after_na = df_meteo[col].isna().sum()
    print(f"  - {col}: {before_na} -> {after_na} NaN (interpoles: {before_na - after_na})")

# Forward fill pour precipitation_mm => (Variables cumulatives)
before_na = (df_meteo['precipitation_mm'].isna() | (df_meteo['precipitation_mm'] == '')).sum()
df_meteo['precipitation_mm'] = df_meteo['precipitation_mm'].replace('', np.nan)
df_meteo['precipitation_mm'] = df_meteo.groupby('commune')['precipitation_mm'].transform(
    lambda x: x.ffill().bfill()
)
after_na = df_meteo['precipitation_mm'].isna().sum()
print(f"  - precipitation_mm: {before_na} -> {after_na} NaN (forward filled)")

[4/6] Traitement des valeurs manquantes ...
  - temperature_c: 3206 -> 0 NaN (interpoles: 3206)


  - humidite_pct: 0 -> 0 NaN (interpoles: 0)
  - rayonnement_solaire_wm2: 0 -> 0 NaN (interpoles: 0)
  - vitesse_vent_kmh: 0 -> 0 NaN (interpoles: 0)


  - precipitation_mm: 0 -> 0 NaN (forward filled)


In [14]:
# --- Déduplication (sur commune,timestamp)
print("[5/6] Déduplication (sur commune, timestamp) ...")
nbr_lignes_av_dedup = len(df_meteo)
df_meteo = df_meteo.drop_duplicates(
    subset=["commune", "ts_h"],
    keep="first"
)
nbr_lignes_supprimees = nbr_lignes_av_dedup - len(df_meteo)
print("    Nombre de lignes supprimées après déduplication : ", nbr_lignes_supprimees)



[5/6] Déduplication (sur commune, timestamp) ...
    Nombre de lignes supprimées après déduplication :  16380


In [15]:
# --- 5. Ajouter des colonnes temporelles
print("[6/6] Ajout des colonnes temporelles ...")

df_meteo['jour'] = df_meteo['ts_h'].dt.date
df_meteo['mois'] = df_meteo['ts_h'].dt.month
df_meteo['saison'] = df_meteo['mois'].map({
    12: 'Hiver', 1: 'Hiver', 2: 'Hiver',
    3: 'Printemps', 4: 'Printemps', 5: 'Printemps',
    6: 'Ete', 7: 'Ete', 8: 'Ete',
    9: 'Automne', 10: 'Automne', 11: 'Automne'
})

df_meteo['jour_de_semaine'] = df_meteo['ts_h'].dt.dayofweek

print("  - Colonnes ajoutees: jour, mois, saison, jour_de_semaine")

print("  - Appercu des données meteo nettoyées : ")
df_meteo.head(10)

[6/6] Ajout des colonnes temporelles ...


  - Colonnes ajoutees: jour, mois, saison, jour_de_semaine
  - Appercu des données meteo nettoyées : 


,commune,ts_h,temperature_c,humidite_pct,rayonnement_solaire_wm2,vitesse_vent_kmh,precipitation_mm,jour,mois,saison,jour_de_semaine
242272,Bordeaux,2023-01-01 00:00:00,8.5,43.9,43.9,0.2,0.0,2023-01-01,1,Hiver,6
233980,Bordeaux,2023-01-01 01:00:00,2.7,39.7,39.7,21.5,5.3,2023-01-01,1,Hiver,6
108665,Bordeaux,2023-01-01 02:00:00,6.2,78.9,78.9,13.1,0.0,2023-01-01,1,Hiver,6
5904,Bordeaux,2023-01-01 03:00:00,10.3,64.2,64.2,0.6,14.7,2023-01-01,1,Hiver,6
23544,Bordeaux,2023-01-01 04:00:00,0.9,36.4,36.4,35.6,0.0,2023-01-01,1,Hiver,6
89966,Bordeaux,2023-01-01 05:00:00,7.9,66.0,66.0,0.3,0.0,2023-01-01,1,Hiver,6
168449,Bordeaux,2023-01-01 06:00:00,1.6,40.2,40.2,24.3,0.0,2023-01-01,1,Hiver,6
51384,Bordeaux,2023-01-01 07:00:00,4.9,87.9,87.9,27.9,0.0,2023-01-01,1,Hiver,6
88813,Bordeaux,2023-01-01 08:00:00,8.4,43.4,43.4,38.8,0.1,2023-01-01,1,Hiver,6
30506,Bordeaux,2023-01-01 09:00:00,10.1,83.8,83.8,25.4,0.0,2023-01-01,1,Hiver,6


In [16]:
# --- Rapport apres nettoyage
quality_report(df_meteo, "Meteo Clean")

RAPPORT QUALITE - Meteo Clean
- Lignes: 236,232
- Colonnes: 11


,Colonne,Type,Manquants,Completude %,Uniques
0,commune,str,0,100.0,15
1,ts_h,datetime64[us],0,100.0,17544
2,temperature_c,float64,0,100.0,1083
3,humidite_pct,float64,0,100.0,652
4,rayonnement_solaire_wm2,float64,0,100.0,1094
5,vitesse_vent_kmh,float64,0,100.0,401
6,precipitation_mm,float64,0,100.0,151
7,jour,object,0,100.0,731
8,mois,int32,0,100.0,12
9,saison,str,0,100.0,4


---

## Creation du dataset nettoye output/meteo_clean.csv

In [17]:
# --- Sauvegarde
df_meteo.to_csv(OUT_METEO_CLEAN_CSV, index=False)
print("[ok]: Création avec succès du dataset nettoyé 'output/meteo_clean.csv'")

[ok]: Création avec succès du dataset nettoyé 'output/meteo_clean.csv'


---

## Libérer la mémoire (Optionnel) 

In [18]:

del df_meteo_raw
del df_meteo
del df_temp

---

## (Optionel) enregistrement dans un fichier temporaire du temps d'execution + ressources pour utilisation ultérieure (dans le script run_pipeline_hybride.py ou autres)

In [19]:
temps_execution_04 = time.time() - start_time_04
temps_resources = f"""
    Date : {datetime.now().strftime("%d/%m/%Y %H:%M:%S")} (FR)

    temps_exec_sec={temps_execution_04:.2f}
    ram_gb={ram_available_04:.2f}
    cpu_pct={cpu_available_pct_04:.2f}
    logi_cores={available_logical_04:.1f}
    physi_cores={available_physical_04:.1f}
"""

# Ecrire des données du temps d'execution + ressources dans le fichier TMP_FILE_TXT
TMP_FILE_TXT.write_text(temps_resources, encoding="utf-8")

137